In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
# Module front matter
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from arcgis.gis import GIS
from arcgis.geocoding import geocode

In [33]:
# Data-oriented front matter
api_key = "AAPK3946a1297c814c71a56299387cc980eexUoR9U9brolhXFwcVbK5mv-Dn3KAJoq-vua_VJTDKyFfqKPYTn2Iy4nwcwGxTFy7"
portal = GIS("https://www.arcgis.com", api_key=api_key)

df = pd.read_csv("athousandtweets.csv")
stop_words = set(stopwords.words('english'))
small_df = df.iloc[:, 1:5]

In [37]:
def master_geocoder(tweet):
    
    tweet_timestamp = tweet[0]
    tweet_density = tweet[1]
    tweet_text = tweet[3]
    
    word_tokens = tweet_text.split()

    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

    filtered_sentence = []

    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)


    short_sentence = ' '.join(filtered_sentence)

    punc = '''!()[]{};:'"\,<>./?@#$%^&*_~'''

    for ele in short_sentence:
        if ele in punc:
            short_sentence = short_sentence.replace(ele, " ")


    short_sentence = short_sentence.replace('Sq', 'Square')
    short_sentence = short_sentence.replace(' St ', ' Street ')
    short_sentence = short_sentence.replace(' SB ', ' Southbound ')
    short_sentence = short_sentence.replace(' EB ', ' Eastbound ')
    short_sentence = short_sentence.replace(' NB ', ' Northbound ')
    short_sentence = short_sentence.replace(' WB ', ' Westbound ')
    short_sentence = short_sentence.replace(' Ave ', ' Avenue ')
    short_sentence = short_sentence.replace(' Rd ', ' Road ')
    short_sentence = short_sentence.replace(' Vehicle ', ' ')
    short_sentence = short_sentence.replace(' vehicle ', ' ')
    short_sentence = short_sentence.replace(' Car ', ' ')
    short_sentence = short_sentence.replace(' car ', ' ')
    short_sentence = short_sentence.replace(' Clear ', ' ')
    short_sentence = short_sentence.replace(' clear ', ' ')
    short_sentence = short_sentence.replace(' Blocked ', ' ')
    short_sentence = short_sentence.replace(' blocked ', ' ')
    short_sentence = short_sentence.replace(' delays ', ' ')
    short_sentence = short_sentence.replace(' may ', ' ')
    short_sentence = short_sentence.replace(' exist ', ' ')
    short_sentence = short_sentence.replace(' Incident ', ' ')
    short_sentence = short_sentence.replace(' CLEARED ', ' ')
    short_sentence = short_sentence.replace(' Construction ', ' ')
    short_sentence = short_sentence.replace(' lane ', ' ')
    short_sentence = short_sentence.replace(' Right ', ' ')
    short_sentence = short_sentence.replace(' accident ', ' ')
    short_sentence = short_sentence.replace(' fire ', ' ')
    short_sentence = short_sentence.replace(' Updated ', ' ')
    short_sentence = short_sentence.replace(' reopened ', ' ')
    short_sentence = short_sentence.replace(' expect ', ' ')
    short_sentence = short_sentence.replace(' Expect ', ' ')
    short_sentence = short_sentence.replace(' CLine ', ' ')
    short_sentence = short_sentence.replace(' NLine ', ' ')
    short_sentence = short_sentence.replace(' accident ', ' ')
    short_sentence = short_sentence.replace(' RLine ', ' ')
    short_sentence = short_sentence.replace(' ALine ', ' ')
    
    
    if len(short_sentence) > 200:
        short_sentence = short_sentence[:169]
    
    
    if len(short_sentence) > 170:
        short_sentence = short_sentence[:169]
    
    
    geocode_results = geocode(address=short_sentence+" New York City")
    
    results_df = pd.DataFrame(columns=["Coords", "City"])
    for i in range(len(geocode_results)):
        # Get coordinates
        new_value = list(geocode_results[i].values())
        coords = list(new_value[1].values())


        # Get city
        loc_details = list(new_value[3].values())
        city = loc_details[37]

        results_df = results_df.append({'Coords' : coords, 'City' : city}, ignore_index = True)

    results_df = results_df[results_df['City'] == "New York City Metro"]
    
    results_df["Tweet"] = tweet_text
    results_df["Timestamp"] = tweet_timestamp
    results_df["Density"] = tweet_density
    
    return(results_df)

In [38]:
master_df = pd.DataFrame()

for i in range(len(small_df)):
    if small_df.isnull().iloc[i, 1]:
        pass
    else:
        master_df = master_df.append(master_geocoder(small_df.iloc[i]))

In [39]:
master_df

,Coords,City,Tweet,Timestamp,Density
0,"[-74.00713999999994, 40.71455000000003]",New York City Metro,CLEARED BUFFALO: DELAY Delays: WB on I-90 exit...,2022-03-29 17:12:13+00:00,0
1,"[-73.88927432395744, 40.82075459501255]",New York City Metro,CLEARED BUFFALO: DELAY Delays: WB on I-90 exit...,2022-03-29 17:12:13+00:00,0
0,"[-74.00713999999994, 40.71455000000003]",New York City Metro,"In #NassauCounty, westbound #SouthernStatePark...",2022-03-29 17:02:00+00:00,1
0,"[-74.00713999999994, 40.71455000000003]",New York City Metro,UPDATE BUFFALO: DELAY Delays: WB on I-90 exit ...,2022-03-29 16:55:54+00:00,2
1,"[-73.88927432395744, 40.82075459501255]",New York City Metro,UPDATE BUFFALO: DELAY Delays: WB on I-90 exit ...,2022-03-29 16:55:54+00:00,2
...,...,...,...,...,...
15,"[-73.87810460240206, 40.904771243342246]",New York City Metro,CLEARED LOWER HUDSON VALLEY: Delays: SB on I-8...,2022-03-28 01:17:11+00:00,0
16,"[-73.83422043721797, 40.87259271137714]",New York City Metro,CLEARED LOWER HUDSON VALLEY: Delays: SB on I-8...,2022-03-28 01:17:11+00:00,0
17,"[-73.816093431475, 40.88776239461886]",New York City Metro,CLEARED LOWER HUDSON VALLEY: Delays: SB on I-8...,2022-03-28 01:17:11+00:00,0
18,"[-73.87628818597302, 40.644158568830015]",New York City Metro,CLEARED LOWER HUDSON VALLEY: Delays: SB on I-8...,2022-03-28 01:17:11+00:00,0


In [40]:
master_df.to_csv("coords_density_df.csv")